## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/model_fold_1.h5
/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/model_fold_2.h5
/kaggle/input/142-googleq-a-train-3fold-bert-base-unc-hidden11/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
#     h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
    h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
#     h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
#     h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))

    x = GlobalAveragePooling1D()(h11)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.939675,0.707987,0.248410,0.585088,0.528851,0.553885,0.670881,0.645579,0.424994,...,0.923393,0.933392,0.673208,0.970467,0.967167,0.845513,0.039534,0.042307,0.887796,0.932233
1,46,0.880806,0.531395,0.007815,0.779337,0.818808,0.914446,0.551433,0.426173,0.080260,...,0.714634,0.948734,0.630691,0.967807,0.973487,0.887781,0.903613,0.129080,0.093376,0.894544
2,70,0.907412,0.705834,0.029093,0.803578,0.855563,0.922683,0.627918,0.465539,0.118938,...,0.861099,0.950789,0.650319,0.969514,0.972510,0.850505,0.082537,0.063644,0.887510,0.922139
3,132,0.905325,0.485051,0.008893,0.761589,0.739244,0.913249,0.559593,0.429674,0.073535,...,0.697867,0.960063,0.692513,0.973658,0.983495,0.908478,0.807610,0.153088,0.530440,0.897300
4,200,0.929200,0.482685,0.033980,0.849687,0.721378,0.881824,0.681056,0.550911,0.115676,...,0.734366,0.916743,0.636582,0.968803,0.968095,0.850559,0.275795,0.097098,0.690188,0.903695


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.908364,0.629164,0.039810,0.762522,0.800282,0.859038,0.593845,0.473406,0.210597,...,0.800521,0.944671,0.667220,0.967266,0.972803,0.880859,0.500840,0.131923,0.538891,0.913323
std,2812.670060,0.030933,0.117908,0.068478,0.089713,0.098954,0.116645,0.049234,0.081214,0.184453,...,0.080551,0.019462,0.047128,0.009130,0.009556,0.035961,0.322346,0.058766,0.287782,0.020172
min,39.000000,0.798134,0.408692,0.004104,0.244260,0.258295,0.263844,0.494992,0.314330,0.012011,...,0.591731,0.844719,0.541351,0.924709,0.932570,0.740120,0.005303,0.009403,0.031858,0.820849
25%,2572.000000,0.889276,0.524713,0.009732,0.725044,0.758433,0.844601,0.554843,0.418804,0.071907,...,0.735554,0.936601,0.633555,0.963490,0.967900,0.863809,0.156513,0.087214,0.284377,0.903491
50%,5093.000000,0.908919,0.629044,0.015234,0.775128,0.808324,0.896083,0.590500,0.446320,0.135928,...,0.807447,0.948437,0.670331,0.968862,0.974779,0.887796,0.584068,0.132057,0.551099,0.916440
75%,7482.000000,0.931411,0.726092,0.032432,0.816563,0.865826,0.929481,0.625096,0.516371,0.299970,...,0.869392,0.957503,0.700116,0.973256,0.979589,0.905231,0.799833,0.173302,0.799722,0.926826
max,9640.000000,0.970198,0.873661,0.654446,0.948829,0.976999,0.972013,0.750096,0.735547,0.899021,...,0.948572,0.981416,0.781235,0.984795,0.989780,0.960991,0.947102,0.312546,0.991254,0.955662
